订购量$\times$(1-亏损比)=销售量

单个商品获利 = 7月1日定价 $\times$ 7月1日订购量$\times$(1-亏损比) - 7月1日批发价 $\times$ 7月1日订购量

In [74]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

In [75]:
def result(file):
    df = pd.read_csv(file, encoding='gbk')

    # 计算平均利润
    df['平均利润'] = (df['销售单价(元/千克)'] - df['批发价格(元/千克)']) * df['销量(千克)']
    # 计算平均利润率
    df['平均利润率'] = (df['销售单价(元/千克)'] - df['批发价格(元/千克)']) / df['批发价格(元/千克)']
    # 计算加成率
    df['加成率'] = df['平均利润率'] / (1 - df['平均利润率'])
    # 计算成本加成定价
    df['成本加成定价'] = df['批发价格(元/千克)'] * (1 + df['加成率'])

    # 如果成本加成定价大于批发价的1.75倍，则将成本加成定价设置为批发价的1.75倍
    max_allowed_price = df['批发价格(元/千克)'] * 1.75
    df['成本加成定价'] = df['成本加成定价'].where(df['成本加成定价'] <= max_allowed_price, max_allowed_price)

    df = df.drop(["平均利润", "平均利润率", "加成率"], axis=1)
    df['日期'] = pd.to_datetime(df['日期'])
    df = df.resample('D', on='日期').agg({'销量(千克)': 'sum', '成本加成定价': 'mean', '销售单价(元/千克)': 'mean', "批发价格(元/千克)": 'mean'})   # ------------------------注意啊这里是以日为单位而不是月
    df['日期'] = df.index

    df['成本加成定价'].fillna(0, inplace=True)
    # 将DataFrame中的所有inf值替换为0
    df.replace([np.inf, -np.inf], 0, inplace=True)

    model = IsolationForest()
    df['is_outlier'] = model.fit_predict(df[['成本加成定价']])
    df_cleaned = df[df['is_outlier'] != -1]

    return df_cleaned



In [76]:
df01 = result("data_3\\白玉菇(袋).csv")
df02 = result("data_3\\菠菜.csv")
df03 = result("data_3\\菠菜(份).csv")
df04 = result("data_3\\菜心.csv")
df05 = result("data_3\\虫草花(份).csv")
df06 = result("data_3\\高瓜(1).csv")
df07 = result("data_3\\高瓜(2).csv")
df08 = result("data_3\\海鲜菇(包).csv")
df09 = result("data_3\\红椒(2).csv")
df10 = result("data_3\\红莲藕带.csv")
df11 = result("data_3\\红薯尖.csv")
df12 = result("data_3\\洪湖藕带.csv")
df13 = result("data_3\\姜蒜小米椒组合装(小份).csv")
df14 = result("data_3\\金针菇(盒).csv")
df15 = result("data_3\\净藕(1).csv")
df16 = result("data_3\\菱角.csv")
df17 = result("data_3\\螺丝椒.csv")
df18 = result("data_3\\螺丝椒(份).csv")
df19 = result("data_3\\木耳菜.csv")
df20 = result("data_3\\木耳菜(份).csv")
df21 = result("data_3\\奶白菜.csv")
df22 = result("data_3\\七彩椒(2).csv")
df23 = result("data_3\\青红杭椒组合装(份).csv")
df24 = result("data_3\\青茄子(1).csv")
df25 = result("data_3\\青线椒(份).csv")
df26 = result("data_3\\上海青.csv")
df27 = result("data_3\\双孢菇(盒).csv")
df28 = result("data_3\\娃娃菜.csv")
df29 = result("data_3\\外地茼蒿.csv")
df30 = result("data_3\\芜湖青椒(1).csv")
df31 = result("data_3\\西兰花.csv")
df32 = result("data_3\\西峡花菇(1).csv")
df33 = result("data_3\\鲜木耳(份).csv")
df34 = result("data_3\\苋菜.csv")
df35 = result("data_3\\小米椒(份).csv")
df36 = result("data_3\\小青菜(1).csv")
df37 = result("data_3\\小皱皮(份).csv")
df38 = result("data_3\\蟹味菇与白玉菇双拼(盒).csv")
df39 = result("data_3\\野生粉藕.csv")
df40 = result("data_3\\圆茄子(2).csv")
df41 = result("data_3\\云南生菜.csv")
df42 = result("data_3\\云南生菜(份).csv")
df43 = result("data_3\\云南油麦菜.csv")
df44 = result("data_3\\云南油麦菜(份).csv")
df45 = result("data_3\\长线茄.csv")
df46 = result("data_3\\枝江青梗散花.csv")
df47 = result("data_3\\竹叶菜.csv")
df48 = result("data_3\\紫茄子(1).csv")
df49 = result("data_3\\紫茄子(2).csv")

In [77]:
df49

,销量(千克),成本加成定价,销售单价(元/千克),批发价格(元/千克),日期,is_outlier
日期,,,,,,
2023-06-24,13.573,6.65,6.0,3.8,2023-06-24,1
2023-06-25,16.448,6.65,6.0,3.8,2023-06-25,1
2023-06-26,16.395,6.65,6.0,3.8,2023-06-26,1
2023-06-27,4.095,6.65,6.0,3.8,2023-06-27,1
2023-06-28,6.075,6.65,6.0,3.8,2023-06-28,1
2023-06-29,5.369,6.65,6.0,3.8,2023-06-29,1


In [78]:
def find_yueshu(df):

    xiaoliang = [min(df["销量(千克)"]), max(df["销量(千克)"])]
    chengbendingjia = [min(df["成本加成定价"]), max(df["成本加成定价"])]
    print(xiaoliang)
    print(chengbendingjia)
    return xiaoliang, chengbendingjia

In [79]:
xiaoliang1, chengbendingjia1 = find_yueshu(df01)


[1.0, 1.0]
[4.192401315789473, 6.2475]


In [80]:
xiaoliang2, chengbendingjia2 = find_yueshu(df02)


[2.293, 3.252]
[16.852500000000003, 16.852500000000003]


In [81]:
xiaoliang3, chengbendingjia3 = find_yueshu(df03)


[2.0, 15.0]
[4.96454081632653, 6.023490099009901]


In [82]:
xiaoliang4, chengbendingjia4 = find_yueshu(df04)


[0.822, 4.112]
[6.587777777777778, 6.6125]


In [83]:
xiaoliang5, chengbendingjia5 = find_yueshu(df05)


[1.0, 4.0]
[4.55, 4.654999999999999]


In [84]:
xiaoliang6, chengbendingjia6 = find_yueshu(df06)


[2.357, 3.477]
[15.264971694971695, 17.52156458339888]


In [85]:
xiaoliang7, chengbendingjia7 = find_yueshu(df07)


[0.512, 2.365]
[15.255044020542918, 18.311368305209097]


In [86]:
xiaoliang8, chengbendingjia8 = find_yueshu(df08)


[4.0, 11.0]
[2.611607142857143, 3.4299999999999997]


In [87]:
xiaoliang9, chengbendingjia9 = find_yueshu(df09)


[1.325, 3.95]
[20.689278634361234, 21.54109446459549]


In [88]:
xiaoliang10, chengbendingjia10 = find_yueshu(df10)


[0.218, 0.862]
[9.9225, 9.957500000000001]


In [89]:
xiaoliang11, chengbendingjia11 = find_yueshu(df11)


[3.0869999999999997, 4.011]
[6.3, 6.3]


In [90]:
xiaoliang12, chengbendingjia12 = find_yueshu(df12)


[4.339, 4.98]
[20.305900621118013, 29.758695652173913]


In [91]:
xiaoliang13, chengbendingjia13 = find_yueshu(df13)


[5.0, 9.0]
[-23.836818181818238, 4.6025]


In [92]:
xiaoliang14, chengbendingjia14 = find_yueshu(df14)


[9.0, 26.0]
[2.2271520053926523, 2.3361111111111112]


In [93]:
xiaoliang15, chengbendingjia15 = find_yueshu(df15)


[4.492999999999999, 7.4030000000000005]
[16.66944899927659, 18.07142857142857]


In [94]:
xiaoliang16, chengbendingjia16 = find_yueshu(df16)


[0.903, 3.396]
[16.03, 16.0825]


In [95]:
xiaoliang17, chengbendingjia17 = find_yueshu(df17)


[4.324, 8.47]
[13.37, 13.57669491525424]


In [96]:
xiaoliang18, chengbendingjia18 = find_yueshu(df18)


[7.0, 19.0]
[4.380051020408162, 6.601962512413108]


In [97]:
xiaoliang19, chengbendingjia19 = find_yueshu(df19)


[2.769, 9.466]
[5.022, 5.431842672413794]


In [98]:
xiaoliang20, chengbendingjia20 = find_yueshu(df20)


[0.0, 1.0]
[-2.7867857142857138, 0.0]


In [99]:
xiaoliang21, chengbendingjia21 = find_yueshu(df21)


[6.34, 9.044]
[-16.26777777777776, 4.7775]


In [100]:
xiaoliang22, chengbendingjia22 = find_yueshu(df22)


[0.168, 1.506]
[19.068006700167505, 21.227500000000003]


In [101]:
xiaoliang23, chengbendingjia23 = find_yueshu(df23)


[1.0, 4.0]
[5.1625000000000005, 5.81]


In [102]:
xiaoliang24, chengbendingjia24 = find_yueshu(df24)


[3.758, 7.098]
[7.21, 7.21]


In [103]:
xiaoliang25, chengbendingjia25 = find_yueshu(df25)


[1.0, 1.0]
[4.4624999999999995, 5.4425]


In [104]:
xiaoliang26, chengbendingjia26 = find_yueshu(df26)


[1.42, 5.553]
[7.140000000000001, 7.140000000000001]


In [105]:
xiaoliang27, chengbendingjia27 = find_yueshu(df27)


[7.0, 13.0]
[5.479230769230769, 5.603728448275862]


In [106]:
xiaoliang28, chengbendingjia28 = find_yueshu(df28)


[6.0, 19.0]
[7.651117429730513, 8.049999999999999]


In [107]:
xiaoliang29, chengbendingjia29 = find_yueshu(df29)


[0.996, 3.68]
[13.405000000000001, 16.0825]


In [108]:
xiaoliang30, chengbendingjia30 = find_yueshu(df30)


[10.334, 17.095]
[5.95, 6.352499999999999]


In [109]:
xiaoliang31, chengbendingjia31 = find_yueshu(df31)


[8.083, 16.9]
[12.985, 13.895000000000001]


In [110]:
xiaoliang32, chengbendingjia32 = find_yueshu(df32)


ValueError: min() arg is an empty sequence

In [ ]:
df32

,销量(千克),成本加成定价,销售单价(元/千克),批发价格(元/千克),日期,is_outlier
日期,,,,,,


In [111]:
xiaoliang33, chengbendingjia33 = find_yueshu(df33)


[4.0, 4.0]
[1.1555555555555557, 1.1555555555555557]


In [112]:
xiaoliang34, chengbendingjia34 = find_yueshu(df34)


[3.809, 11.061]
[4.077500000000001, 4.1475]


In [113]:
xiaoliang35, chengbendingjia35 = find_yueshu(df35)


[18.0, 23.0]
[-3.2252739726027393, -2.1690808823529406]


In [114]:
xiaoliang36, chengbendingjia36 = find_yueshu(df36)


[4.788, 5.97]
[5.25, 5.25]


In [115]:
xiaoliang37, chengbendingjia37 = find_yueshu(df37)


[6.0, 18.0]
[2.255948275862069, 2.7697560975609754]


In [116]:
xiaoliang38, chengbendingjia38 = find_yueshu(df38)


[3.0, 3.0]
[5.180355029585798, 5.180355029585798]


In [117]:
xiaoliang39, chengbendingjia39 = find_yueshu(df39)


[0.231, 1.441]
[28.104999999999997, 28.122500000000002]


In [118]:
xiaoliang40, chengbendingjia40 = find_yueshu(df40)


[0.553, 2.209]
[5.6000000000000005, 7.7700000000000005]


In [121]:
xiaoliang41, chengbendingjia41 = find_yueshu(df41)


[0.251, 7.971]
[10.045, 10.08]


In [122]:
xiaoliang42, chengbendingjia42 = find_yueshu(df42)


[23.0, 58.0]
[4.214895470383276, 4.866660493938239]


In [123]:
xiaoliang43, chengbendingjia43 = find_yueshu(df43)


[0.573, 4.232]
[7.840000000000001, 7.840000000000001]


In [126]:
xiaoliang44, chengbendingjia44 = find_yueshu(df44)


[18.0, 26.0]
[4.53289310232102, 4.958316400580552]


In [129]:
xiaoliang45, chengbendingjia45 = find_yueshu(df45)


[2.332, 8.789]
[12.2325, 12.2675]


In [130]:
xiaoliang46, chengbendingjia46 = find_yueshu(df46)


[2.161, 2.668]
[16.803333333333335, 16.92250865051903]


In [131]:
xiaoliang47, chengbendingjia47 = find_yueshu(df47)


[12.022, 16.569]
[4.1475, 4.234999999999999]


In [132]:
xiaoliang48, chengbendingjia48 = find_yueshu(df48)
xiaoliang49, chengbendingjia49 = find_yueshu(df49)

[0.512, 0.512]
[-58.8612499999999, -58.8612499999999]
[4.095, 16.448]
[6.6499999999999995, 6.65]


In [133]:
xiaoliang49, chengbendingjia49

([4.095, 16.448], [6.6499999999999995, 6.65])